<a href="https://colab.research.google.com/github/kake01/Bot/blob/master/musuka.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [0]:
#ドライブをマウントする
from google.colab import drive
drive.mount('/content/drive')

In [0]:
# 蛇の目インストール
pip install janome

In [0]:
# テキストデータの分割
import sys
import os
import chardet
import tensorflow as tf
import os.path
import pprint

from os.path import join, splitext, split

# 入力ディレクトリと出力ディレクトリを指定
readdir = sys.argv[1]
outdir ="drive/My Drive/Bot/Serif"

# 天空の城ラピュタのテキストデータを代入
file_data = open('drive/My Drive/Bot/TextData.txt', 'r',encoding='utf-8')
lines = file_data.readlines()
# #読み込んだテキストファイルを１行ずつ表示
# for line in lines:

# キャラ名を取得
    char_name = line[:line.find(u"「")]
    outfname = os.path.join(outdir, char_name + ".txt")
#キャラ名のファイルがあるかどうか確認
    if os.path.exists(outfname):
        # ある場合は上書きモードで
        outfp = open(outfname, "a")
    else:
        # ない場合は新規作成
        outfp = open(outfname, "w")
        
    # セリフのみ抽出
    line_format = line[line.find(u"「") + 1:line.find(u"」")] + "\n"
    # セリフ書き込み
    outfp.write(line_format)

In [0]:
# 関数関係
from collections import Counter
from collections import defaultdict
import random
from janome.tokenizer import Tokenizer

BEGIN = '__BEGIN__'
END = '__END__'

""" 文章を3単語の組にして返す"""
def get_three_words_list(sentence):
  t = Tokenizer()
  words = t.tokenize(sentence, wakati=True)
  words = [BEGIN] + words + [END]
  three_words_list = []
  for i in range(len(words) - 2):
    three_words_list.append(tuple(words[i:i+3]))

  return three_words_list

""" マルコス連鎖での文章生成用の辞書データを生成する """
def generate_markov_dict(three_words_conut):
  markov_dict = {}
  for three_words, count in three_words_conut.items():
    two_words = three_words[:2]
    next_word = three_words[2]
    if two_words not in markov_dict:
      markov_dict[two_words] = {'words': [], 'weights': []}
    markov_dict[two_words]['words'].append(next_word)
    markov_dict[two_words]['weights'].append(count)
  
  return markov_dict

""" 最初の単語を選択するための辞書データを作成する """
def get_first_word_and_count(three_words_conut):

  first_word_count = defaultdict(int)

  for three_words, count in three_words_conut.items():
    if three_words[0] == BEGIN:
      next_word = three_words[1]
      first_word_count[next_word] += count

  return first_word_count

""" 最初の単語を選択するための辞書データを作成する """
def get_first_word_and_count(three_words_conut):
  first_word_count = defaultdict(int)

  for three_words, count in three_words_conut.items():
    if three_words[0] == BEGIN:
      next_word = three_words[1]
      first_word_count[next_word] += count

  return first_word_count

""" 最初の単語と重みのリストを作成する　"""
def get_first_words_weights(three_words_conut):
  first_word_count = get_first_word_and_count(three_words_conut)
  words = []
  weights = []
  for word, count in first_word_count.items():
    words.append(word)
    weights.append(count)

  return words, weights

""" 入力された辞書データを元に文章を生成する """
def generate_text(first_words, first_weights, markov_dict):
  first_word = random.choices(first_words, weights = first_weights)[0]
  generate_words = [BEGIN, first_word]
  while True:
    pair = tuple(generate_words[-2:])
    words = markov_dict[pair]['words']
    weights = markov_dict[pair]['weights']
    next_word = random.choices(words, weights=weights)[0]
    if next_word == END:
      break
    generate_words.append(next_word)

  return ''.join(generate_words[1:])
  

In [0]:
# テキストデータの前処理
# 1文章を配列に入れる
import time
from tqdm import tqdm
import sys
import os
import chardet
import os.path
import pprint
import re
from os.path import join, splitext, split

sentences1 = []
three_words_list = []

path = 'drive/My Drive/Bot/Serif/ムスカ.txt'
file_data = open(path, 'r',encoding='utf-8')

# 1文章を配列に入れる
for line in file_data:
  line_format = line[:line.find("。")]
  # sentences1.append(line_format)




for sentence in tqdm(sentences1):
  three_words_list += get_three_words_list(sentence)
three_words_conut = Counter(three_words_list)

# ムスカ生成
markov_dict1 = generate_markov_dict(three_words_conut)
first_words, first_weights = get_first_words_weights(three_words_conut)

100%|██████████| 125/125 [00:08<00:00, 14.94it/s]


In [0]:
print(sentences1)

['くいとめろ', 'てこずらせたな...', '制服さんの悪い癖だ', '閣下が不用意に打たれた暗号を解読されたのです', 'これは、私の機関の仕事です', 'もちろん', 'よく眠れたかな？', 'はやりの服は嫌いですか？...彼なら安心したまえ', '入りたまえ', 'すさまじい破壊力を持つ、ロボットの兵隊だよ', 'こいつは...、地上で造られたものではない', 'そこだ', '同じ印が、君の家の古い暖炉にあった', 'こいつは君の手にあるときにしか、働かない', '石は持ち主を守り、いつの日にか天空のラピュタへ帰るときの道しるべとして君に受け継がれたんだ', '君は、ラピュタを宝島か何かのように考えているのかね...', 'ラピュタはかつて、恐るべき科学力で天空にあり、全地上を支配した、恐怖の帝国だったのだ!!', '...そんなものがまだ空中をさまよっているとしたら、平和にとってどれだけ危険なことか、君にも分かるだろう', '私に協力して欲しい', '私は、手荒なことはしたくないが...あの少年の運命は君がにぎっているんだよ...', '君が協力してくれるんなら、あの少年を自由の身にしてやれるんだ', 'ウルはラピュタ語で『王』', 'パズー君', 'ラピュタの調査は、シータさんの協力で、軍が極秘に行うことになったんだ', '君も男なら、ききわけたまえ!!!!', 'これはわずかだが、心ばかりのお礼だ', '思いだしたまえ', '約束さえ果たせば、君も自由になれる', 'もう少し、時間がいります', 'すばらしい', '古文書にあった通りだ', '...ほぁぉあ!!!!', 'どんな呪文だ!!教えろその言葉を!!!!', 'ロボット？', 'ここへ来る気か？', 'すごい...!!!!', 'そぉかぁ...', 'その光だ', 'ああ、', 'ああ、ああ、ああー', 'あ゛あ゛あ゛ーーーっ、ああ゛っっ!!!!', 'あああー、あぁーー!!', '飛ぶ気か!!!!？', 'うああっっ!!!!!!', 'ああっ!!', 'あの光のさす方向に、ラピュタがあるのだ!!!!...、まだか!!早くしろ!!!!', '私はムスカ大佐だ', 'おおあ!!!!', 'どけぇ!!!!...しまった...', 'くそぉ、ゴリアテ、何をしてる!!!!', 'っあ...', '煙

In [0]:
# print(sentences1)
# Janomeのロード
from janome.tokenizer import Tokenizer

# Tokenneizerインスタンスの生成 
t = Tokenizer()

# テキストを引数として、形態素解析の結果、名詞・動詞・形容詞の原形のみを配列で抽出する関数を定義 
def extract_words(text):
    tokens = t.tokenize(text)
    return [token.base_form for token in tokens 
        if token.part_of_speech.split(',')[0] in['名詞', '動詞','形容詞']]

# 分かち書きの配列生成
for line in file_data:
  line_format = line[:line.find("。")]
  word_list = [extract_words(sentence) for sentence in line_format]

# 結果の一部を確認
print(word_list)

In [0]:
# 文字列を生成する
for _ in range(10):
  text = generate_text(first_words, first_weights, markov_dict1)
  print(text)